In [3]:
import numpy as np 
import pandas as pd


In [4]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')


In [5]:
movies = movies.merge(credits, on = 'title') 

In [6]:
movies = movies[['genres' , 'movie_id' , 'keywords' , 'title' , 'overview' , 'cast' , 'crew']]

In [7]:
movies.isnull().sum()

genres      0
movie_id    0
keywords    0
title       0
overview    3
cast        0
crew        0
dtype: int64

In [8]:
movies.dropna(inplace = True)

In [9]:
movies.duplicated().sum()

0

In [10]:
import ast 


In [11]:
def convert(obj):
    L =[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
     
        
    

In [12]:
movies['genres'] = movies['genres'].apply(convert)

In [13]:
movies['keywords'] = movies['keywords'].apply(convert)

In [14]:
def convert_2(obj):
    L =[]
    counter = 0 
    for i in ast.literal_eval(obj):
        if counter!= 5:
           L.append(i['name'])
           counter += 1 
        else :
            break ;
    return L
     

In [15]:
movies['cast'] = movies['cast'].apply(convert_2)

In [16]:
def convert_director(obj):
    L =[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director' :
             L.append(i['name'])
             break ;
    return L
     

In [17]:
 movies['crew'] = movies['crew'].apply(convert_director)

In [18]:
movies['overview'] = movies['overview'].apply(lambda x : x.split() )

In [19]:
movies['genres'] = movies['genres'].apply(lambda x :[i.replace(" " , "") for i in x ])
movies['cast'] = movies['cast'].apply(lambda x :[i.replace(" " , "") for i in x ])
movies['crew'] = movies['crew'].apply(lambda x :[i.replace(" " , "") for i in x ])
movies['keywords'] = movies['keywords'].apply(lambda x :[i.replace(" " , "") for i in x ])

In [20]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [21]:
new_movies = movies[['movie_id' , 'title' , 'tags']]

In [22]:
new_movies['tags'] = new_movies['tags'].apply(lambda x :" ".join(x))

<ipython-input-22-62a0829ff2a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(lambda x :" ".join(x))


In [23]:
new_movies['tags'] = new_movies['tags'].apply(lambda x : x.lower())

<ipython-input-23-d2a3162e0aac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(lambda x : x.lower())


In [24]:
import nltk 

In [25]:
!pip install nltk

<class 'OSError'>: Not available

In [26]:
from nltk.stem.porter import PorterStemmer

In [27]:
ps = PorterStemmer()

In [28]:
def stem(text):
    y =[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [29]:
new_movies ['tags'] = new_movies['tags'].apply(stem)

<ipython-input-29-38115c1442c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies ['tags'] = new_movies['tags'].apply(stem)


In [30]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 7000 , stop_words = 'english')

In [31]:
vectors = cv.fit_transform(new_movies['tags']).toarray()

In [32]:
from sklearn.metrics.pairwise import cosine_similarity 

In [ ]:
similarity = cosine_similarity(vectors)

In [33]:
def recommend(movie):
    movie_index = new_movies[new_movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    answer = sorted(list(enumerate(distances)) , reverse = True , key = lambda x : x[1])[1:6]
    for i in answer :
        print(new_movies.iloc[i[0]].title)
        

In [1]:
import pickle

In [2]:
pickle.dump(new_movies.to_dict() , open('movies_dict.pkl' , 'wb'))

<class 'NameError'>: name 'new_movies' is not defined

In [ ]:
pickle.dump(similarity , open('similarity.pkl'  , 'wb')) 